In [1]:
# Random Forest Classification
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

C:\Users\M1051327\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train0 = train[train.project_is_approved == 0]
train1 = train[train.project_is_approved == 1]

In [4]:
print(train0.shape)
print(train1.shape)

(27734, 16)
(154346, 16)


In [5]:
# Separate majority and minority classes
df_majority = train1[train1.project_is_approved==1]
df_minority = train0[train0.project_is_approved==0]

In [6]:
df_majority.shape

(154346, 16)

In [7]:
df_minority.shape

(27734, 16)

In [8]:
from sklearn.utils import resample
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=154346,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.project_is_approved.value_counts()

1    154346
0    154346
Name: project_is_approved, dtype: int64

In [9]:
df_upsampled.shape

(308692, 16)

In [ ]:
list(df_upsampled)

In [10]:
train = train[['teacher_number_of_previously_posted_projects','teacher_prefix','school_state','project_grade_category',
               'project_is_approved']]

In [11]:
df_upsampled = df_upsampled[['teacher_number_of_previously_posted_projects','teacher_prefix','school_state','project_grade_category',
               'project_is_approved']]

In [12]:
test = test[['teacher_number_of_previously_posted_projects','teacher_prefix','school_state','project_grade_category']]

In [13]:
df_upsampled.head()

,teacher_number_of_previously_posted_projects,teacher_prefix,school_state,project_grade_category,project_is_approved
0,26,Ms.,NV,Grades PreK-2,1
2,5,Ms.,UT,Grades 3-5,1
4,42,Mr.,CA,Grades 6-8,1
5,0,Mrs.,DE,Grades PreK-2,1
6,1,Mrs.,MO,Grades PreK-2,1


In [14]:
# Import required packages
from sklearn import preprocessing

# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [15]:
df_upsampled['teacher_prefix'] = le.fit_transform(df_upsampled['teacher_prefix'].astype(str))
df_upsampled['school_state'] = le.fit_transform(df_upsampled['school_state'].astype(str))
df_upsampled['project_grade_category'] = le.fit_transform(df_upsampled['project_grade_category'].astype(str))

In [16]:
train['teacher_prefix'] = le.fit_transform(train['teacher_prefix'].astype(str))

In [17]:
train['school_state'] = le.fit_transform(train['school_state'].astype(str))

In [18]:
train['project_grade_category'] = le.fit_transform(train['project_grade_category'].astype(str))

In [ ]:
#test = test[['teacher_number_of_previously_posted_projects']]

In [19]:
test['teacher_prefix'] = le.fit_transform(test['teacher_prefix'].astype(str))
test['school_state'] = le.fit_transform(test['school_state'].astype(str))
test['project_grade_category'] = le.fit_transform(test['project_grade_category'].astype(str))

In [ ]:
test.head()

In [20]:
X = df_upsampled.iloc[:, [0,1,2,3]].values
y = df_upsampled.iloc[:, [4]].values

In [21]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [22]:

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


C:\Users\M1051327\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\M1051327\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\M1051327\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:

# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

C:\Users\M1051327\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [24]:
# Predicting the Test set results
y_pred_prob = classifier.predict(X_test)

In [25]:
probabilities = classifier.predict_proba(test)

In [26]:
probabilities = pd.DataFrame(probabilities)

In [27]:
probabilities.head()

,0,1
0,0.1000,0.9000
1,0.4000,0.6000
2,0.4875,0.5125
3,0.4875,0.5125
4,0.1000,0.9000


In [28]:
del probabilities[0]

In [29]:
test['probabilities'] = probabilities

In [30]:
test.tail()

,teacher_number_of_previously_posted_projects,teacher_prefix,school_state,project_grade_category,probabilities
78030,14,2,1,3,1.0000
78031,1,3,32,3,0.7000
78032,4,2,4,1,0.9000
78033,9,2,37,0,0.6000
78034,0,2,15,0,0.6375


In [ ]:
test.to_csv("main_test_set_results.csv",index=True)

In [31]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_prob)
cm

array([[28748,  9705],
       [19996, 18724]], dtype=int64)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_prob))

              precision    recall  f1-score   support

           0       0.59      0.75      0.66     38453
           1       0.66      0.48      0.56     38720

   micro avg       0.62      0.62      0.62     77173
   macro avg       0.62      0.62      0.61     77173
weighted avg       0.62      0.62      0.61     77173



In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_prob)*100

61.513741852720514